In [ ]:
import pydeck as pyd
import pandas as pd

# Plotting lights at night

NASA has collected global light emission data for over 30 years. The data set is a deeply fascinating one and has been used for news stories on the Syrian Civil War [[1]](https://time.com/3741451/syria-lights-civil-war-satellite/), North Korea [[2]](https://www.cbsnews.com/pictures/north-korea-hermit-country-space-photos/4/), and economic growth [[3]](https://qz.com/959563/nasas-black-marble-map-shows-the-light-of-human-population-centers-at-night-throughout-2016/).

In this notebook, we'll use a deck.gl [ScreenGridLayer](https://deck.gl/#/examples/core-layers/screen-grid-layer) to visualize some of the changes at different points in time.

## Getting the data

The data for Chengdu, China, is cleaned and available below:

In [ ]:
LIGHTS_URL = 'https://raw.githubusercontent.com/ajduberstein/lights_at_night/master/chengdu_lights_at_night.csv'
df = pd.read_csv(LIGHTS_URL)
df.head()

### Setting the colors
pydeck does need to know the color for this data in advance of plotting it

In [ ]:
df['color'] = df['brightness'].apply(lambda val: [255, val * 4,  255, 255])
df.sample(10)

### Configuring the coordinates

Currently pydeck expects coordinates to be an array listed in one field, which we can implement in Pandas:

In [ ]:
df['position'] = df.apply(lambda row: [row['lng'], row['lat']], axis=1)
# Make the data frame smaller by only plotting useful fields
result_df = df[['position', 'color', 'year']]
result_df.head()

## Plotting and interacting

We can plot this data set of light brightness by year, configuring a slider to filter the data as below:

In [ ]:
plottable = result_df[result_df['year'] == 1993].to_dict(orient='records')

view_state = pyd.ViewState(latitude=31.0, longitude=104.5, zoom=8)
scatterplot = pyd.Layer(
    'ScatterplotLayer',
    data=plottable,
    get_position='position',
    get_fill_color='color',
    opacity=0.5,
    get_radius=800)
r = pyd.Deck(layers=[scatterplot], initial_view_state=view_state)
r.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
slider = widgets.IntSlider(1992, min=1993, max=2013, step=2)
def on_change(v):
    plottable = result_df[result_df['year'] == slider.value].to_dict(orient='records')
    scatterplot.data = plottable
    r.update()
    
slider.observe(on_change, names='value')
display(slider)